#Breast Cancer

Predict whether a tumor is malignant or benign.

##About Dataset

###Description:

Breast cancer is the most prevalent cancer among women globally, accounting for 25% of all cancer cases. In 2015 alone, it impacted over 2.1 million individuals. The disease begins when cells in the breast grow uncontrollably, forming tumors that can be detected via X-ray or felt as lumps.


The primary challenge in its detection is classifying tumors as malignant (cancerous) or benign (non-cancerous). We invite you to analyze and classify these tumors using machine learning techniques, specifically Support Vector Machines (SVMs), with the Breast Cancer Wisconsin (Diagnostic) Dataset.


###Acknowledgements:

This dataset is sourced from Kaggle.

###Objective:

Understand and clean the dataset if necessary.
Build classification models to predict if the cancer is malignant or benign.
Fine-tune hyperparameters and compare the performance of various classification algorithms.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rahmasleam/breast-cancer")

print("Path to dataset files:", path)

C:\Users\fabri\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 48.6k/48.6k [00:00<00:00, 12.8MB/s]

Extracting files...
Path to dataset files: C:\Users\fabri\.cache\kagglehub\datasets\rahmasleam\breast-cancer\versions\1


Carregamento e limpeza dos dados

Lemos o arquivo CSV com pandas (pd.read_csv(...)).
Exploramos o DataFrame usando métodos como df.head() e df.info().
Removemos a coluna id, pois geralmente não possui valor preditivo para o diagnóstico.
Checamos se há valores nulos e, caso existam, decidimos como tratá-los (ex.: apagar linhas, preencher com média etc.).

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix



df = pd.read_csv(r"C:\Users\fabri\.cache\kagglehub\datasets\rahmasleam\breast-cancer\versions\1\breast-cancer.csv")

df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [14]:
df.drop('id', axis=1, inplace=True)

In [16]:
print(df.isnull().sum())

diagnosis                  0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64


Transformar a coluna de diagnóstico

Convertemos a coluna diagnosis para valores numéricos, onde “M” = 1 (maligno) e “B” = 0 (benigno). Isso facilita o uso de modelos de classificação.

In [17]:
df['diagnosis'].value_counts()

diagnosis
B    357
M    212
Name: count, dtype: int64

In [18]:
df['diagnosis']=df['diagnosis'].map({'B': 0, 'M' : 1})

Separar o conjunto de dados em X (features) e y (target)

X contém todas as colunas de características (ex.: radius_mean, perimeter_mean, etc.).
y contém apenas a coluna-alvo (diagnosis).

In [19]:
x = df.drop('diagnosis', axis=1)
y = df['diagnosis']

Dividir em treino e teste

Utilizamos train_test_split para dividir os dados em um conjunto de treino (para ajustar o modelo) e um conjunto de teste (para avaliar o modelo em dados “desconhecidos”).
test_size=0.3 significa que 30% dos dados serão usados para teste.
random_state=42 apenas garante reprodutibilidade (opcional).
stratify=y garante que a proporção de classes (M vs B) se mantém equilibrada nos dois conjuntos.

In [35]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=42, stratify=y)

Escalonar (padronizar) os dados

Alguns algoritmos (especialmente aqueles baseados em distância ou que fazem otimização por gradiente, como a Regressão Logística) têm desempenho melhor quando as variáveis estão na mesma escala.
O StandardScaler transforma cada feature para que tenha média 0 e desvio padrão 1.

In [34]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Treinar um modelo de classificação

Regressão Logística: É frequentemente um dos primeiros modelos testados, fácil de interpretar e muitas vezes apresenta bons resultados para esse tipo de problema.
Random Forest: Algoritmo de ensemble que costuma ter boa performance em problemas com muitas features.

Avaliar o modelo

A acurácia (accuracy_score) mede a porcentagem de classificações corretas.
A matriz de confusão (confusion_matrix) mostra quantos acertos e erros existem para cada classe.
O relatório de classificação (classification_report) exibe métricas de precisão (precision), revocação (recall) e pontuação F1, úteis para avaliar o desempenho principalmente em casos de classe desbalanceada.

In [ ]:
# 7) Treinar um modelo de classificação
#    Exemplo 1: Regressão Logística

logreg = LogisticRegression(max_iter=10000)
logreg.fit(X_train_scaled, y_train)

y_pred_logreg = logreg.predict(X_test_scaled)

# Avaliação do modelo de Regressão Logística
print('\n=== Regressão Logística ===')
print('Acurácia no teste: ', accuracy_score(y_test, y_pred_logreg))
print('Matriz de confusão:\n', confusion_matrix(y_test, y_pred_logreg))
print('Relatório de classificação:\n', classification_report(y_test, y_pred_logreg))


=== Regressão Logística ===
Acurácia no teste:  0.9707602339181286
Matriz de confusão:
 [[106   1]
 [  4  60]]
Relatório de classificação:
               precision    recall  f1-score   support

           0       0.96      0.99      0.98       107
           1       0.98      0.94      0.96        64

    accuracy                           0.97       171
   macro avg       0.97      0.96      0.97       171
weighted avg       0.97      0.97      0.97       171



In [ ]:
#    Exemplo 2: Random Forest

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)

y_pred_rf = rf.predict(X_test_scaled)

# Avaliação do modelo de Random Forest
print('\n=== Random Forest ===')
print('Acurácia no teste: ', accuracy_score(y_test, y_pred_rf))
print('Matriz de confusão:\n', confusion_matrix(y_test, y_pred_rf))
print('Relatório de classificação:\n', classification_report(y_test, y_pred_rf))


=== Random Forest ===
Acurácia no teste:  0.9649122807017544
Matriz de confusão:
 [[107   0]
 [  6  58]]
Relatório de classificação:
               precision    recall  f1-score   support

           0       0.95      1.00      0.97       107
           1       1.00      0.91      0.95        64

    accuracy                           0.96       171
   macro avg       0.97      0.95      0.96       171
weighted avg       0.97      0.96      0.96       171



Previsão de apenas 1 amostra do próprio conjunto de teste
Suponha que você queira prever o diagnóstico para a primeira amostra do conjunto de teste (X_test) e comparar com o valor real (y_test).

In [36]:
# 1) Pegamos uma única amostra do X_test
amostra_unica = X_test.iloc[0]  # Series com as features

# 2) É preciso escalonar essa amostra, pois o modelo foi treinado em dados escalonados
amostra_unica_escalonada = scaler.transform([amostra_unica])  # transforma em array 2D

# 3) Fazer a predição
predicao = logreg.predict(amostra_unica_escalonada)

print("Predição (0 = Benigno, 1 = Maligno): ", predicao[0])

# 4) Comparar com o valor real em y_test
print("Valor real: ", y_test.iloc[0])

Predição (0 = Benigno, 1 = Maligno):  0
Valor real:  0


C:\Users\fabri\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Previsão de um caso novo (fora do dataset)
Imagine que você tenha os valores de uma paciente que não está no dataset e queira fazer a predição. Suponha que sejam 30 colunas (excluindo o diagnosis), e que estejam na mesma ordem e nome das colunas que você usou para treinar.

In [37]:
# Exemplo FICTÍCIO de valores de entrada, apenas ilustrativo
# Certifique-se de que a ordem das colunas corresponde à usada no treino.
# Por exemplo: [radius_mean, texture_mean, perimeter_mean, ..., fractal_dimension_worst]
valores_novos = [
    15.50,  20.20,  120.9,  800.0,  0.100,  0.200,  0.150,  0.050, 
    0.160,  0.055,  0.800,  0.900,  5.000,  85.50,  0.007,  0.040,
    0.045,  0.010,  0.035,  0.008,  22.0,   25.0,   150.0,  1200.0,
    0.150,  0.250,  0.300,  0.150,  0.300,  0.080
]

# Vamos criar um DataFrame com 1 linha, usando exatamente as colunas de X
df_novo = pd.DataFrame([valores_novos], columns=x.columns)

# Escalonar
df_novo_scaled = scaler.transform(df_novo)

# Fazer a predição
pred_novo = logreg.predict(df_novo_scaled)

# Ver probabilidade de cada classe
prob_novo = logreg.predict_proba(df_novo_scaled)

print("Predição (0 = Benigno, 1 = Maligno):", pred_novo[0])
print("Probabilidades [Classe 0, Classe 1]:", prob_novo[0])


Predição (0 = Benigno, 1 = Maligno): 1
Probabilidades [Classe 0, Classe 1]: [0.00555341 0.99444659]
